In [73]:
from music21 import *
from collections import defaultdict, OrderedDict
from itertools import groupby
from grammar import *


In [84]:
def __parse_midi(data_fn):
    # Parse the MIDI data for separate melody and accompaniment parts.
    midi_data = converter.parse(data_fn)
    # Get melody part, compress into single voice.
    melody_stream = midi_data[5]     # For Metheny piece, Melody is Part #5.
    melody1, melody2 = melody_stream.getElementsByClass(stream.Voice)
    for j in melody2:
        melody1.insert(j.offset, j)
    melody_voice = melody1

    for i in melody_voice:
        if i.quarterLength == 0.0:
            i.quarterLength = 0.25

    # Change key signature to adhere to comp_stream (1 sharp, mode = major).
    # Also add Electric Guitar. 
    melody_voice.insert(0, instrument.ElectricGuitar())
    melody_voice.insert(0, key.KeySignature(sharps=1))

    # The accompaniment parts. Take only the best subset of parts from
    # the original data. Maybe add more parts, hand-add valid instruments.
    # Should add least add a string part (for sparse solos).
    # Verified are good parts: 0, 1, 6, 7 '''
    partIndices = [0, 1, 6, 7]
    comp_stream = stream.Voice()
    comp_stream.append([j.flat for i, j in enumerate(midi_data) 
        if i in partIndices])

    # Full stream containing both the melody and the accompaniment. 
    # All parts are flattened. 
    full_stream = stream.Voice()
    for i in range(len(comp_stream)):
        full_stream.append(comp_stream[i])
    full_stream.append(melody_voice)

    # Extract solo stream, assuming you know the positions ..ByOffset(i, j).
    # Note that for different instruments (with stream.flat), you NEED to use
    # stream.Part(), not stream.Voice().
    # Accompanied solo is in range [478, 548)
    solo_stream = stream.Voice()
    for part in full_stream:
        curr_part = stream.Part()
        curr_part.append(part.getElementsByClass(instrument.Instrument))
        curr_part.append(part.getElementsByClass(tempo.MetronomeMark))
        curr_part.append(part.getElementsByClass(key.KeySignature))
        curr_part.append(part.getElementsByClass(meter.TimeSignature))
        curr_part.append(part.getElementsByOffset(476, 548, 
                                                  includeEndBoundary=True))
        cp = curr_part.flat
        solo_stream.insert(cp)

    # Group by measure so you can classify. 
    # Note that measure 0 is for the time signature, metronome, etc. which have
    # an offset of 0.0.
    melody_stream = solo_stream[-1]
    measures = OrderedDict()
    offsetTuples = [(int(n.offset / 4), n) for n in melody_stream]
    measureNum = 0 # for now, don't use real m. nums (119, 120)
    for key_x, group in groupby(offsetTuples, lambda x: x[0]):
        measures[measureNum] = [n[1] for n in group]
        measureNum += 1

    # Get the stream of chords.
    # offsetTuples_chords: group chords by measure number.
    chordStream = solo_stream[0]
    chordStream.removeByClass(note.Rest)
    chordStream.removeByClass(note.Note)
    offsetTuples_chords = [(int(n.offset / 4), n) for n in chordStream]

    # Generate the chord structure. Use just track 1 (piano) since it is
    # the only instrument that has chords. 
    # Group into 4s, just like before. 
    chords = OrderedDict()
    measureNum = 0
    for key_x, group in groupby(offsetTuples_chords, lambda x: x[0]):
        chords[measureNum] = [n[1] for n in group]
        measureNum += 1

    # Fix for the below problem.
    #   1) Find out why len(measures) != len(chords).
    #   ANSWER: resolves at end but melody ends 1/16 before last measure so doesn't
    #           actually show up, while the accompaniment's beat 1 right after does.
    #           Actually on second thought: melody/comp start on Ab, and resolve to
    #           the same key (Ab) so could actually just cut out last measure to loop.
    #           Decided: just cut out the last measure. 
    del chords[len(chords) - 1]
    assert len(chords) == len(measures)

    return measures, chords

''' Helper function to get the grammatical data from given musical data. '''
def __get_abstract_grammars(measures, chords):
    # extract grammars
    abstract_grammars = []
    for ix in range(1, len(measures)):
        m = stream.Voice()
        for i in measures[ix]:
            m.insert(i.offset, i)
        c = stream.Voice()
        for j in chords[ix]:
            c.insert(j.offset, j)
        parsed = parse_melody(m, c)
        abstract_grammars.append(parsed)

    return abstract_grammars


In [75]:
measures, chords = __parse_midi("../data/original_metheny.mid")
abstract_grammars = __get_abstract_grammars(measures, chords)

In [86]:
midi_data = converter.parse("../data/original_metheny.mid")

# Get melody part, compress into single voice.
melody_stream = midi_data[5]     # For Metheny piece, Melody is Part #5.
melody1, melody2 = melody_stream.getElementsByClass(stream.Voice)
for j in melody2:
    melody1.insert(j.offset, j)
melody_voice = melody1

for i in melody_voice:
    if i.quarterLength == 0.0:
        i.quarterLength = 0.25

midi_data.show('text')

{0.0} <music21.stream.Part 0x7f5e9f5ea828>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.tempo.MetronomeMark Quarter=112.0>
    {0.0} <music21.key.Key of G major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x7f5e9f5d36d8>
        {0.0} <music21.note.Rest rest>
        {4.0} <music21.chord.Chord C4 A3 E4 G3 F2>
        {5.75} <music21.chord.Chord E4 C4 A3 G3 F2>
        {6.5} <music21.chord.Chord F3 B-3 A3 D4>
        {8.0} <music21.chord.Chord D2 C4 E3 F3 A3>
        {10.5} <music21.chord.Chord B-2 F4 C4 D4 A4>
        {11.0} <music21.chord.Chord B-3 E-3 F4 D4 G3>
        {12.5} <music21.chord.Chord C4 E4 A3 F3 D2>
        {13.75} <music21.chord.Chord F3 A3 E4 C4 D2>
        {14.5} <music21.chord.Chord B-3 F4 D4 A3>
        {20.0} <music21.chord.Chord C4 A3 E4 G3 F2>
        {21.75} <music21.chord.Chord E4 C4 A3 G3 F2>
        {22.5} <music21.chord.Chord F3 B-3 A3 D4>
        {24.0} <music21.chord.Chord D2 C4 E3 F3 A3>
        {26.5} <mu

In [87]:
my_data = converter.parse("../data/time.mid")
my_data.show('text')

{0.0} <music21.stream.Part 0x7f5e9b7e7ba8>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.tempo.MetronomeMark allegretto Quarter=110.0>
    {0.0} <music21.note.Rest rest>
    {225.0} <music21.note.Note A>
    {225.5} <music21.note.Note E>
    {226.0} <music21.note.Note A>
    {226.5} <music21.note.Note B>
    {227.0} <music21.note.Note B>
    {227.5} <music21.note.Note E>
    {228.0} <music21.note.Note A>
    {228.5} <music21.note.Note B>
    {229.0} <music21.note.Note B>
    {229.5} <music21.note.Note E>
    {230.0} <music21.note.Note A>
    {230.5} <music21.note.Note B>
    {231.0} <music21.note.Note B>
    {231.5} <music21.note.Note E>
    {232.0} <music21.note.Rest rest>
    {233.0} <music21.note.Note A>
    {233.5} <music21.note.Note E>
    {234.0} <music21.note.Note A>
    {234.5} <music21.note.Note B>
    {235.0} <music21.note.Note B>
    {235.5} <music21.note.Note E>
    {236.0}